# Regresión Lineal Pesada
### #Introducción, objetivo, y contexto#

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

import sys
sys.path.append('././src')

from lwlr import LW_LinReg, LW_LogReg

In [ ]:
# Cargamos los datos
Data = pd.read_csv("././data/loess.csv")

# Separamos los features y la variable objetivo
X = Data["X"].values.reshape(-1, 1) # Aseguramos que X tenga la forma correcta
y = Data["y"].values

# Dividimos los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Probamos el modelo de regresión lineal pesada con diferentes valores de tau
taus = [0.005, 0.01, 0.1, 1, 10, 50]
predictions = {tau: LW_LinReg(X_train, y_train, X_test, tau=tau) for tau in taus}

In [ ]:
# Graficamos los resultados
plt.figure(figsize=(15, 10))

plt.scatter(X_test, y_test, color='black', label='Real', alpha=0.7)
for tau, y_pred in predictions.items():
    plt.scatter(X_test, y_pred, label=f"tau={tau}", alpha=0.5)

plt.title("Predicciones de Regresión Lineal Pesada con Diferentes Valores de Tau")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Calculamos el MSE para cada valor de tau
print("\nMSE en test para cada tau:")
for tau, y_pred in predictions.items():
    mse = mean_squared_error(y_test, y_pred)
    print(f"τ = {tau:<5}  ->  MSE: {mse:.4f}")

In [ ]:
# Cargamos los datos de clasificación
Data = pd.read_csv("././data/dataset_clasificacion.csv")

# Separamos los features y la variable objetivo
X = Data[["X1", "X2"]].values
y = Data["Clase"].values

# Dividimos los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## original, luego se cambia

In [ ]:
# Función para graficar la frontera
def plot_lwlr_decision_boundary(X_train, y_train, tau, title):
    h = 0.3  # resolución baja para acelerar
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    grid = np.c_[xx.ravel(), yy.ravel()]

    # Tomar muestra de puntos
    np.random.seed(0)
    sample_indices = np.random.choice(len(grid), size=200, replace=False)
    grid_sampled = grid[sample_indices]
    Z_sampled = LW_LogReg(X_train, y_train, grid, tau, max_iter=100, lr=0.01)

    # Malla con NaN y rellenar los puntos calculados
    Z_full = np.full(xx.shape, np.nan)
    for idx, (x_val, y_val) in enumerate(grid):
        i = int((y_val - y_min) / h) 
        j = int((x_val - x_min) / h)
        if 0 <= i < Z_full.shape[0] and 0 <= j < Z_full.shape[1]:
            Z_full[i, j] = Z_sampled[idx]
      
    # Graficamos
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z_full, alpha=0.4, cmap='RdBu')
    plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolor='k', cmap='RdBu', alpha=0.7)
    plt.title(title)
    plt.xlabel("X1")
    plt.ylabel("X2")
    plt.tight_layout()
    plt.show()


# Ejecutar para distintos τ
taus = [0.01, 0.1, 1, 10]
for tau in taus:
    plot_lwlr_decision_boundary(X_train, y_train, tau, title=f"Frontera de Decisión - τ = {tau}")